In [118]:
sys.path.insert(0, '/home/matheus/Documentos/house-recommendation/src')
import pandas as pd
from joblib import load
from model_inference import ModelInference
from model_training import ModelTraining
from experiments import Experiments
from data_source import DataSource
from metrics import Metrics

In [2]:
ModelInference().predict()

Loading model
Loading data
Preprocessing Data
Creating dataframe for data manipulation
Droping columns with missing values
Dropping column with id
Creating list with numeric features
Creating list with categorical features
removing target
['bairro']
feature encoder
feature normalization and encoding
bairro       0
area         0
quartos      0
garagem      0
banheiros    0
preco        0
dtype: int64
Predicting
Evaluating model in test data
{'roc_auc_score': 0.8451502732240438, 'average_precision_score': 0.6234080141331406}
Saving file


array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [2]:
modelo = load('../output/modelo.pkl')

In [5]:
modelo

{'model': LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 'preprocessing': <preprocessing.PreProcessing at 0x7f120fe91880>,
 'columns': ['area', 'quartos', 'garagem', 'banheiros', 'preco', 'bairro']}

In [6]:
encoder = modelo['preprocessing'].catb
numeric = modelo['preprocessing'].scaler

In [7]:
features_names = modelo['preprocessing'].feature_names

In [8]:
entrada_dados = {
    'Id' : 132,
    'area' : 44,
    'quartos' : 2,
    'garagem' : 1,
    'banheiros' : 1,
    'bairro' : 'Aparecida',
    'preco' : 180000
}

In [9]:
df_teste = pd.DataFrame(index=[0], columns=features_names)
df_teste = df_teste.fillna(value=0)

for i in entrada_dados.items():
  df_teste[i[0]] = i[1]

In [10]:
df_teste

,area,quartos,garagem,banheiros,preco,bairro,Id
0,44,2,1,1,180000,Aparecida,132


In [12]:
df_teste.drop('Id', axis = 1, inplace = True)

In [13]:
numeric_features = df_teste.drop('bairro', axis = 1).columns.tolist()
df_teste['bairro'] = encoder.transform(df_teste['bairro'])
df_teste[numeric_features] = numeric.transform(df_teste[numeric_features])

In [14]:
df_teste

,area,quartos,garagem,banheiros,preco,bairro
0,-0.669648,-0.837344,-0.76879,-1.047397,-0.63686,0.170833


In [18]:
modelo['model'].predict(df_teste)

array([0])

## Verificando imbalanced

In [15]:
import functools
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import scipy.stats as sct
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

In [16]:
df = pd.read_csv('/home/matheus/Documentos/house-recommendation/data/train.csv', delimiter = ';')

In [17]:
df.head()

,Id,area,quartos,garagem,banheiros,bairro,preco,y
0,1718,600,4,3,4,Morada Da Colina,700000,0
1,528,61,2,1,1,Vida Nova,170000,0
2,1649,58,2,2,2,Pampulha,226000,0
3,1719,50,2,1,2,Novo Mundo,185000,1
4,42,78,2,2,3,Tabajaras,330000,0


In [27]:
df.y.value_counts()

0    995
1    205
Name: y, dtype: int64

In [18]:
df['bairro'] = encoder.transform(df['bairro'])
df[numeric_features] = numeric.transform(df[numeric_features])

In [19]:
df.head()

,Id,area,quartos,garagem,banheiros,bairro,preco,y
0,1718,2.529816,1.435987,0.877148,1.777045,0.010049,0.912903,0
1,528,-0.571823,-0.837344,-0.768790,-1.047397,0.017083,-0.666664,0
2,1649,-0.589086,-0.837344,0.054179,-0.105917,0.098674,-0.499766,0
3,1719,-0.635121,-0.837344,-0.768790,-0.105917,0.634167,-0.621959,1
4,42,-0.473997,-0.837344,0.054179,0.835564,0.017083,-0.189813,0


In [114]:
df = pd.read_csv('/home/matheus/Documentos/house-recommendation/data/train.csv', delimiter = ';')
df['bairro'] = encoder.transform(df['bairro'])
df[numeric_features] = numeric.transform(df[numeric_features])

In [115]:
df.y.value_counts()

0    995
1    205
Name: y, dtype: int64

In [117]:
smote = SMOTE(sampling_strategy=0.1)
X = df.drop('y', axis=1)
y = df['y'].values
X_smote, y_smote = smote.fit_resample(X, y)

ValueError: The specified ratio required to remove samples from the minority class while trying to generate new samples. Please increase the ratio.

In [81]:
df_smote = X_smote
df_smote['y'] = y_smote
df_smote

,Id,area,quartos,garagem,banheiros,bairro,preco,y
0,1718,2.529816,1.435987,0.877148,1.777045,0.010049,0.912903,0
1,528,-0.571823,-0.837344,-0.768790,-1.047397,0.017083,-0.666664,0
2,1649,-0.589086,-0.837344,0.054179,-0.105917,0.098674,-0.499766,0
3,1719,-0.635121,-0.837344,-0.768790,-0.105917,0.634167,-0.621959,1
4,42,-0.473997,-0.837344,0.054179,0.835564,0.017083,-0.189813,0
...,...,...,...,...,...,...,...,...
1985,756,-0.602025,-0.837344,-0.768790,-1.047397,0.373792,-0.580996,1
1986,377,-0.581099,-0.028327,-0.768790,-0.377303,0.316893,-0.524223,1
1987,1225,-0.569242,-0.037096,-0.768790,-0.105917,0.280817,-0.719996,1
1988,759,-0.617777,-0.837344,-0.768790,-0.119203,0.460779,-0.577969,1


In [82]:
df_smote.y.value_counts()

1    995
0    995
Name: y, dtype: int64

In [83]:
df_smote.drop('Id', axis = 1, inplace = True)

In [87]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_smote, test_size=0.33, random_state=42, shuffle=True)

In [88]:
X = train.drop('y', axis = 1)
y = train['y'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)

In [89]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [91]:
y_pred = model.predict(X_val)

In [93]:
Metrics().calculate_classification(y_val, y_pred)

{'roc_auc_score': 0.9368782445037332,
 'average_precision_score': 0.8981548001846261}

In [95]:
y_pred_test = model.predict(test.drop('y', axis = 1))


In [96]:
y_test = test['y'].values

In [97]:
Metrics().calculate_classification(y_test, y_pred_test)

{'roc_auc_score': 0.956452573214751,
 'average_precision_score': 0.9375919409121212}

In [98]:
entrada_dados = {
    'Id' : 132,
    'area' : 44,
    'quartos' : 2,
    'garagem' : 1,
    'banheiros' : 1,
    'bairro' : 'Aparecida',
    'preco' : 180000
}

In [99]:
df_teste = pd.DataFrame(index=[0], columns=features_names)
df_teste = df_teste.fillna(value=0)

for i in entrada_dados.items():
  df_teste[i[0]] = i[1]

In [100]:
df_teste

,area,quartos,garagem,banheiros,preco,bairro,Id
0,44,2,1,1,180000,Aparecida,132


In [101]:
df_teste.drop('Id', axis = 1, inplace = True)

In [102]:
numeric_features = df_teste.drop('bairro', axis = 1).columns.tolist()
df_teste['bairro'] = encoder.transform(df_teste['bairro'])
df_teste[numeric_features] = numeric.transform(df_teste[numeric_features])

In [103]:
df_teste

,area,quartos,garagem,banheiros,preco,bairro
0,-0.669648,-0.837344,-0.76879,-1.047397,-0.63686,0.077238


In [104]:
model.predict(df_teste)

array([0])